<a href="https://colab.research.google.com/github/jsilveira1409/CIVIL-459-Animal-Pose-Estimation/blob/sda-dev/openpifpaf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
!pip3 install openpifpaf

import openpifpaf
import IPython
from openpifpaf.transforms import Crop
from PIL import Image
import gdown
import subprocess
openpifpaf.show.Canvas.show = True


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.3/202.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.2/166.2 kB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 69.2 MB/s eta 0:00:00
  Using cached torch-1.13.1-cp39-cp39-manylinux1_x86_64.whl (887.4 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Created wheel for openpifpaf: filename=openpifpaf-0.13.11-c

ModuleNotFoundError: ignored

# Dataset download and manipulation

In [3]:
!rm -rf plugins
!rm -rf openpifpaf_animalplugin
!git clone --branch sda-dev https://github.com/jsilveira1409/CIVIL-459-Animal-Pose-Estimation.git plugins
!mv plugins/openpifpaf_animalplugin/ openpifpaf_animalplugin
!mv plugins/coco_adapt.py coco_adapt.py
from openpifpaf_animalplugin import AnimalPoseEstimation
#from openpifpaf_sdaplugin import SDA
import coco_adapt as coco_adapt

!rm -rf data-animalpose/
!mkdir data-animalpose
!gdown "https://drive.google.com/drive/folders/1xxm6ZjfsDSmv6C9JvbgiGrmHktrUjV5x" -O data-animalpose --folder 
!unzip data-animalpose/images.zip -d data-animalpose/
!rm data-animalpose/images.zip

Cloning into 'plugins'...
remote: Enumerating objects: 1861, done.
remote: Counting objects: 100% (1716/1716), done.
remote: Compressing objects: 100% (1493/1493), done.
remote: Total 1861 (delta 185), reused 1692 (delta 166), pack-reused 145
Receiving objects: 100% (1861/1861), 70.19 MiB | 20.09 MiB/s, done.
Resolving deltas: 100% (230/230), done.
Retrieving folder list
Processing file 1Ge9jZppE9pGxqVyNAh6bWh4_ukffWBlF images.zip
Processing file 1bAidtvR3MttetncXz0mfr47vYy7Uf5hm keypoints.json
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1Ge9jZppE9pGxqVyNAh6bWh4_ukffWBlF
To: /content/data-animalpose/images.zip
100% 368M/368M [00:14<00:00, 26.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bAidtvR3MttetncXz0mfr47vYy7Uf5hm
To: /content/data-animalpose/keypoints.json
100% 2.36M/2.36M [00:00<00:00, 203MB/s]
Download completed
Archive:  data-animalpose/images.zip
   creat

In [4]:
coco_adapt.adapt_to_coco()
config = openpifpaf.plugin.register()
openpifpaf.DATAMODULES

{'animal': openpifpaf.plugins.animalpose.animal_kp.AnimalKp,
 'apollo': openpifpaf.plugins.apollocar3d.apollo_kp.ApolloKp,
 'cifar10': openpifpaf.plugins.cifar10.datamodule.Cifar10,
 'cocodet': openpifpaf.plugins.coco.cocodet.CocoDet,
 'cocokp': openpifpaf.plugins.coco.cocokp.CocoKp,
 'crowdpose': openpifpaf.plugins.crowdpose.module.CrowdPose,
 'nuscenes': openpifpaf.plugins.nuscenes.nuscenes.NuScenes,
 'posetrack2018': openpifpaf.plugins.posetrack.posetrack2018.Posetrack2018,
 'posetrack2017': openpifpaf.plugins.posetrack.posetrack2017.Posetrack2017,
 'cocokpst': openpifpaf.plugins.posetrack.cocokpst.CocoKpSt,
 'wholebody': openpifpaf.plugins.wholebody.wholebody.Wholebody,
 'custom_animal': openpifpaf_animalplugin.dataset.AnimalPoseEstimation}

# Animal Pose Estimation Dataset and Plugin

In [5]:
dataset = AnimalPoseEstimation()

# Semantic Data Augmentation

In [ ]:
sda = SDA(output_dir='data-animalpose/sda')

sda(dataset[0][0], dataset[0][1])

TypeError: 'AnimalPoseEstimation' object is not subscriptable

# Training

In [ ]:
!python3 -m openpifpaf.train \
    --dataset custom_animal \
    --basenet=shufflenetv2k30 \
    --lr=0.00002 \
    --momentum=0.95 \
    --epochs=50 \
    --lr-decay 160 260 \
    --lr-decay-epochs=10  \
    --weight-decay=1e-5 \
    --weight-decay=1e-5 \
    --val-interval 10 \
    --loader-workers 2 \
    --batch-size 5

INFO:__main__:neural network device: cuda (CUDA available: True, count: 1)
INFO:openpifpaf.network.basenetworks:shufflenetv2k30: stride = 16, output features = 2048
INFO:openpifpaf.network.losses.multi_head:multihead loss: ['custom-animal.cif.c', 'custom-animal.cif.vec', 'custom-animal.cif.scales', 'custom-animal.caf.c', 'custom-animal.caf.vec', 'custom-animal.caf.scales'], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
INFO:__main__:Single GPU training
INFO:openpifpaf.logger:{'type': 'process', 'argv': ['/usr/local/lib/python3.9/dist-packages/openpifpaf/train.py', '--dataset', 'custom_animal', '--basenet=shufflenetv2k30', '--lr=0.00002', '--momentum=0.95', '--epochs=50', '--lr-decay', '160', '260', '--lr-decay-epochs=10', '--weight-decay=1e-5', '--weight-decay=1e-5', '--val-interval', '10', '--loader-workers', '2', '--batch-size', '5'], 'args': {'output': 'outputs/shufflenetv2k30-230425-083939-custom_animal.pkl', 'disable_cuda': False, 'ddp': False, 'local_rank': None, 'sync_batchnorm': True, 'quiet'

In [ ]:
import zipfile
from google.colab import drive, files
import os

# create a zip file of the folder you want to save
folder_path = '/outputs'
zip_path = '/content/myfolder.zip'
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            zipf.write(os.path.join(root, file))

# save the zip file to your Google Drive
drive_path = '/content/drive/My Drive/myfolder.zip'
!cp '{zip_path}' '{drive_path}'